In [41]:
import numpy as np

from intrepid_environment.drone_environment import DroneController

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
controller = DroneController()
controller.restart()

Read sensors:

In [40]:
# API Methods

## Read Sensors

# controller.acceleration()
# controller.linear_velocity()
# controller.angular_velocity()
# controller.rotation_angles()

## Control Motors

# controller.control_motors(0.4, 0.7, 0.1, 0.33)

Control drone with a random signal

In [8]:
for _ in range(0, 5000):
  thrust = np.random.uniform(0, 200, 4).tolist()
  try:
    controller.control_motors(*thrust)
    await controller.session_step()
  except Exception as e:
    print("drone crashed")
    break

drone crashed


# Math Helpers

In [ ]:
def skew_symmetric(v):
    """Compute the skew-symmetric matrix of a 3D vector."""
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

def rotation_matrix_i_wrt_b(q):
    """
    Compute the rotation matrix from body frame to inertial frame.
    
    Args:
      q: A numpy auaternion in [w, x, y, z] order.

    Returns:
      A 3x3 numpy array representing the rotation matrix.
    """
    w = q[0]
    v = q[1:4]
    Sk = skew_symmetric(v)
    term1 = np.eye(3) * (w**2 - np.dot(v, v))
    term2 = 2 * np.outer(v, v)
    term3 = 2 * w * Sk

    return term1 + term2 + term3

def constrain(value, lower_bound, upper_bound):
    return max(lower_bound, min(upper_bound, value))

## Body Rate Controller

In [38]:
# Drone Parameters

## Moment of Inertia

Ixx = 1.0
Iyy = 1.0
Izz = 1.0

mass = 1.0

max_tilt_angle = 1.0
max_ascent_rate = 1.0


In [ ]:
# control gain for body_rate_control
kp_PRQ = 0.1

def body_rate_control(pqr_cmd, pqr):
    """
    Calculate a desired 3-axis moment given a desired and current body rate

    Args:
      pqr_cmd: desired body rates [rad/s]
      pqr: current or estimated body rates [rad/s]
    Returns:
      A 3x1 numpy array containing the desired moments for each of the 3 axes.
    """

    global Ixx, Iyy, Izz, kp_PRQ

    I = np.array([Ixx, Iyy, Izz])
    moment_cmd = I * kp_PRQ * (pqr_cmd - pqr)
    return moment_cmd

In [ ]:
kp_Bank = 1.0

def roll_pitch_yaw_control(accel_cmd, attitude, coll_thrust_cmd):
    """
    Calculate a desired pitch and roll angle rates based on a desired global
	    lateral acceleration, the current attitude of the quad, and desired
	    collective thrust command

    Args:
        accel_cmd: desired acceleration in global XY coordinates [m/s2]
        attitude: current or estimated attitude of the vehicle
        coll_thrust_cmd: desired collective thrust of the quad [N]
    Returns:
        A 3x1 numpy array containing the desired pitch and roll rates. The Z
	        element of the V3F should be left at its default value (0)
    """

    global mass, kp_Bank

    pqr_cmd = np.zeros(3)
    R = rotation_matrix_i_wrt_b(attitude)
    if coll_thrust_cmd == 0:
      return pqr_cmd

    coll_accel = -coll_thrust_cmd / mass

    bx_cmd = constrain(accel_cmd[0] / coll_accel, -max_tilt_angle, max_tilt_angle)
    bx_err = bx_cmd - R[0, 2]

    by_cmd = constrain(accel_cmd[1] / coll_accel, -max_tilt_angle, max_tilt_angle)
    by_err = by_cmd - R[1, 2]

    bx_p_term = kp_Bank * bx_err
    by_p_term = kp_Bank * by_err

    R1 = np.array([
       R[1, 0], -R[0, 0],
       R[1, 1], -R[0, 1]
    ]) / R[2, 2]

    pqr_cmd[0] = R1[0, 0] * bx_p_term + R1[0, 1] * by_p_term
    pqr_cmd[1] = R1[1, 0] * bx_p_term + R1[1, 1] * by_p_term

    return pqr_cmd

In [39]:
integrated_altitude_error = 0.0

kp_pos_z = 1.0
kp_vel_z = 1.0
ki_pos_z = 1.0

def altitude_control(pos_z_cmd, vel_z_cmd, pos_z, vel_z, attitude, accel_z_cmd, dt):
    """
	  Calculate desired quad thrust based on altitude setpoint, actual altitude,
	    vertical velocity setpoint, actual vertical velocity, and a vertical 
	    acceleration feed-forward command

    Args:
	    pos_z_cmd, vel_z_cmd: desired vertical position and velocity in NED [m]
	    pos_z, vel_z: current vertical position and velocity in NED [m]
	    accel_z_cmd: feed-forward vertical acceleration in NED [m/s2]
	    dt: the time step of the measurements [seconds]
    Returns:
      A collective thrust command in [N]
    """

    global integrated_altitude_error, kp_pos_z, kp_vel_z, ki_pos_z, max_ascent_rate

    R = rotation_matrix_i_wrt_b(attitude)

    pos_z_err = pos_z_cmd - pos_z
    vel_z_err = vel_z_cmd - vel_z
    integrated_altitude_error += pos_z_err * dt

    b_z = R[2, 2]

    p_term = kp_pos_z * pos_z_err
    d_term = kp_vel_z * vel_z_err
    i_term = ki_pos_z * integrated_altitude_error

    u1_bar = p_term + i_term + d_term + accel_z_cmd

    acc = (u1_bar - 9.81) / b_z
    thrust = -mass * constrain(acc, -max_ascent_rate / dt. max_ascent_rate / dt)

    return thrust

In [ ]:
kp_pos_xy = 1.0
kp_vel_xy = 1.0

def lateral_position_control(pos_cmd, vel_cmd, pos, vel, accel_cmd_ff):
    """
	  Calculate a desired horizontal acceleration based on 
	   desired lateral position/velocity/acceleration and current pose

    Args:
	    pos_cmd: desired position, in NED [m]
	    vel_cmd: desired velocity, in NED [m/s]
	    pos: current position, NED [m]
	    vel: current velocity, NED [m/s]
	    accel_cmd_ff: feed-forward acceleration, NED [m/s2]
    Returns:
      A 3x1 numpy array with desired horizontal accelerations. 
        the Z component should be 0
    """

    global kp_pos_xy, kp_vel_xy

    accel_cmd_ff[2] = 0
    vel_cmd[2] = 0
    pos_cmd[2] = pos[2]

    accel_cmd = accel_cmd_ff

    pos_err = pos_cmd - pos
    vel_err = vel_cmd - vel

    accel = kp_pos_xy * pos_err + kp_vel_xy * vel_err + accel_cmd_ff
    accel_cmd[0] = accel[0]
    accel_cmd[1] = accel[1]


    return accel_cmd

In [ ]:
kp_yaw = 1.0

def yaw_control(yaw_cmd, yaw):
    """
	  Calculate a desired yaw rate to control yaw to yawCmd

    Args:
	    yaw_cmd: commanded yaw [rad]
	    yaw: current yaw [rad]
    Returns:
      a desired yaw rate [rad/s]
    """

    yaw_rate_cmd = kp_yaw * (yaw_cmd - yaw)

    return yaw_rate_cmd

# Test Controllers

In [43]:
# TODO